In [20]:
# Import dependencies
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
import urllib.request
import io
from io import BytesIO
import json
import requests
from config import nrel_key

In [15]:
# Define URL of target table
url = "https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/20100025/en"

In [16]:
# Create Get request to retrieve JSON data from url
url_json = requests.get(url).json()
# Confirm link returned as value with "object" key from get request
archive_link = url_json["object"]
archive_link

'https://www150.statcan.gc.ca/n1/tbl/csv/20100025-eng.zip'

In [4]:
# Extract content from zip archive
# Define access_url for urllib to open archive_link
access_url = urllib.request.urlopen(archive_link)
# Use ZipFile and BytesIO to read contents from access_url
zf = ZipFile(BytesIO(access_url.read()))
# save CSV file variable of target table from zip archive contents
sales_data = zf.read('20100025.csv')


In [5]:
# Save csv file to pandas DataFrame, decode <class 'bytes'> type to csv format
sales_df = pd.read_csv(io.StringIO(sales_data.decode("utf-8")))
sales_df

,REF_DATE,GEO,DGUID,Zero-Emission Vehicles Fuel Type,Vehicle type,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2017-01,Canada,2016A000011124,All zero-emission vehicles,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277479871,1.1.1.1,3488.0,NaN,NaN,NaN,0
1,2017-01,Canada,2016A000011124,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277485216,1.2.1.1,1664.0,NaN,NaN,NaN,0
2,2017-01,Canada,2016A000011124,Plug-in hybrid electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277490561,1.3.1.1,1824.0,NaN,NaN,NaN,0
3,2017-01,Newfoundland and Labrador,2016A000210,All zero-emission vehicles,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277479872,2.1.1.1,NaN,..,NaN,NaN,0
4,2017-01,Newfoundland and Labrador,2016A000210,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277485217,2.2.1.1,NaN,..,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320695,2021-10,Taloyoak,2016A00056208087,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277490559,5345.2.1.1,NaN,..,NaN,NaN,0
320696,2021-10,Taloyoak,2016A00056208087,Plug-in hybrid electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277495904,5345.3.1.1,NaN,..,NaN,NaN,0
320697,2021-10,"Kitikmeot, Unorganized",2016A00056208098,All zero-emission vehicles,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277485215,5346.1.1.1,NaN,..,NaN,NaN,0
320698,2021-10,"Kitikmeot, Unorganized",2016A00056208098,Battery electric,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1277490560,5346.2.1.1,NaN,..,NaN,NaN,0


In [9]:
# Save DataFrame to CSV file in raw_datasets directory
# Define file path to save data
filepath = Path('raw_datasets/EV_sales_data_raw.csv')
# Allow folders and subfolders to be created is needed
filepath.parent.mkdir(parents=True, exist_ok=True)
# Save the file
sales_df.to_csv(filepath)

In [45]:
# Set base URL for Alernative Fuel Stations Call
stations_url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json?&limit=all&country=CA&api_key=" + nrel_key

In [46]:
charging_stations = requests.get(stations_url).json()

In [52]:
total_stations = charging_stations["total_results"]
total_stations

8984

In [55]:
public_stations = [x for x in charging_stations["fuel_stations"] if x["access_code"] == "public"]
len(public_stations)

8539

In [57]:
ontario_stations = [x for x in public_stations if x["state"] == "ON"]
len(ontario_stations)

2384